In [1]:
!gdown --id 1nP49793_P2MyNd2OTk2WXpziU1jEJ3F1
!gdown --id 1ejbbPDiJwu0rBc3pMN6Q70A7ymqHtGcu
!gdown --id 10sE6IhHvAn3w8FPqQdpfnOpQ7wSeura_
!gdown --id 1k1JbP2iSg21PBlwXzvGQTCubFVoOH39y

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1nP49793_P2MyNd2OTk2WXpziU1jEJ3F1
From (redirected): https://drive.google.com/uc?id=1nP49793_P2MyNd2OTk2WXpziU1jEJ3F1&confirm=t&uuid=8023c594-8388-4337-acc9-ae909aa70fa7
To: /content/mt5_xl_22_model.pkl
100% 2.33G/2.33G [00:24<00:00, 94.3MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ejbbPDiJwu0rBc3pMN6Q70A7ymqHtGcu
To: /content/mt5_xl_22tokenizer.pkl
100% 8.33M/8.33M [00:00<00:00, 132MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--i

In [2]:
import pickle
import torch
import pandas as pd


with open('mt5_xl_22_model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('mt5_xl_22tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# model.to(tokenizer)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [3]:
def file_read(path):
    return pd.read_csv(path)

In [4]:
df_val = file_read('Translated_hindi_22_val.csv')
df_test = file_read('Translated_hindi_22_test.csv')

In [5]:
!pip install rouge_score
!pip install rouge


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9b0cb469d86162f0ec1230299a0611ce54e89c18014ea93d4565db2c7f41b837
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [6]:
from rouge import Rouge
def predict_summary(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=600, truncation=True)
    inputs.to("cuda")
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=40, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def score(df):

    df['predicted_summary'] = df['Article'].apply(predict_summary)

    rouge = Rouge()


    def calculate_rouge(row):
        return rouge.get_scores(row['predicted_summary'], row['Summary'], avg=True)

    df['rouge'] = df_test.apply(calculate_rouge, axis=1)
    df['rouge-1'] = df_test['rouge'].apply(lambda x: x['rouge-1']['f'])
    df['rouge-2'] = df_test['rouge'].apply(lambda x: x['rouge-2']['f'])

    print(df['rouge-1'].mean())
    print(df['rouge-2'].mean())

In [7]:
df_test['rouge-l'] = df_test['rouge'].apply(lambda x: x['rouge-l']['f'])
print(df_test['rouge-l'].mean())

KeyError: 'rouge'

In [ ]:
score(df_test)

In [ ]:
score(df_val)

In [ ]:
print("Rouge-1 score :" ,df_test['rouge-1'].mean())
print("Rouge-2 score :", df_test['rouge-2'].mean())
print("Rouge-L score :", df_test['rouge-l'].mean())

Rouge-1 score : 0.5377975666855225
Rouge-2 score : 0.4262612143157365
Rouge-L score : 0.49874335501375167
